In [1]:
import librosa
from SpecAugment import spec_augment_pytorch
import os
from pathlib import Path
from math import ceil
from pydub import AudioSegment

import os
import sys
import numpy as np
import random
from audio import read_mfcc
from batcher import sample_from_mfcc
from constants import SAMPLE_RATE, NUM_FRAMES
from conv_models import DeepSpeakerModel
from test import batch_cosine_similarity
import tensorflow as tf
import logging
import matplotlib.pyplot as plt
from tabulate import tabulate
from torch.utils.data import Dataset

Using TensorFlow backend.


In [31]:
audio_path = '/Users/ethanzh/Code/automatic-speaker-recognition/audio/split/SherlockHolmes/ethan/SherlockHolmes_1.wav'

In [32]:
raw_mfcc = read_mfcc(audio_path, SAMPLE_RATE)
raw_mfcc.shape

(962, 64)

In [33]:
sampled_mfcc = sample_from_mfcc(raw_mfcc, NUM_FRAMES)
sampled_mfcc.shape

(160, 64, 1)

In [18]:
model = DeepSpeakerModel()
model.m.load_weights('ResCNN_triplet_training_checkpoint_265.h5', by_name=True)

In [19]:
model.m.predict(np.expand_dims(raw_mfcc, axis=0))

array([[ 5.29366825e-03, -1.77952331e-02, -6.47019073e-02,
        -3.26307230e-02, -7.83695653e-03, -2.84621934e-03,
         1.09644737e-02,  5.05160876e-02,  2.06758212e-02,
        -9.31135286e-03, -2.25587133e-02,  4.50928276e-03,
         5.54547319e-03, -4.43809554e-02,  4.99752583e-04,
        -3.41944471e-02,  2.44984124e-02, -1.77322906e-02,
         6.41553774e-02, -6.90589547e-02,  7.19904143e-04,
        -2.26973300e-03, -1.24142475e-01, -1.52670378e-02,
         6.69557555e-03,  6.52666017e-02,  4.42101806e-02,
        -3.76745984e-02, -4.45243679e-02,  4.16646898e-02,
        -6.84347078e-02,  2.98913429e-03, -4.44389284e-02,
         5.97076211e-03,  1.85010191e-02, -1.44963926e-02,
         4.62239310e-02, -1.47548849e-02, -1.42725771e-02,
        -3.63146402e-02,  7.52732605e-02, -2.67843734e-02,
        -4.39700261e-02,  2.32001557e-03,  4.53615114e-02,
         4.10564877e-02,  3.30664963e-02, -2.34160461e-02,
        -1.47338537e-02,  1.29114743e-02,  1.73416883e-0

In [3]:
def single_audio_to_mfcc(path):
    mfcc = sample_from_mfcc(read_mfcc(path, SAMPLE_RATE), NUM_FRAMES)
    return mfcc[:, :, 0]
    #predict = model.m.predict(np.expand_dims(mfcc, axis=0))
    new_clip_name = path.replace('wav', 'npy')
    #np.save(new_clip_name, predict)

In [4]:
res = single_audio_to_mfcc(audio_path)

spec_augment_pytorch.visualization_spectrogram(mel_spectrogram=res, title="Raw Mel Spectrogram")

# reshape spectrogram shape to [batch_size, time, frequency]
shape = res.shape
#mel_spectrogram = np.reshape(mel_spectrogram, (-1, shape[0], shape[1]))
res = torch.from_numpy(res)

warped_masked_spectrogram = spec_augment_pytorch.spec_augment(mel_spectrogram=res)

spec_augment_pytorch.visualization_spectrogram(mel_spectrogram=warped_masked_spectrogram, title="pytorch Warped & Masked Mel Spectrogram")

In [25]:
import argparse
import librosa
import numpy as np
import torch
from SpecAugment import spec_augment_pytorch
import os, sys


# Step 0 : load audio file, extract mel spectrogram
audio, sampling_rate = librosa.load(audio_path)
mel_spectrogram = librosa.feature.melspectrogram(y=audio,
                                                    sr=sampling_rate,
                                                    n_mels=128,
                                                    hop_length=128,
                                                    fmax=8000)

# reshape spectrogram shape to [batch_size, time, frequency]
shape = mel_spectrogram.shape
#mel_spectrogram = np.reshape(mel_spectrogram, (-1, shape[0], shape[1]))
mel_spectrogram = torch.from_numpy(mel_spectrogram)

print(shape)
# Show Raw mel-spectrogram
spec_augment_pytorch.visualization_spectrogram(mel_spectrogram=mel_spectrogram, title="Raw Mel Spectrogram")

# Calculate SpecAugment pytorch
#warped_masked_spectrogram = spec_augment_pytorch.spec_augment(mel_spectrogram=mel_spectrogram)

# Show time warped & masked spectrogram
#spec_augment_pytorch.visualization_spectrogram(mel_spectrogram=warped_masked_spectrogram, title="pytorch Warped & Masked Mel Spectrogram")

(128, 17595)


SystemExit: 0